# 交叉熵损失函数

在上一个章节里，我们在网络模型的输出层后使用了 Softmax 激活函数，成功将模型的数值输出转化为“概率分布”。但在实际训练中，我们并没有看到准确率的提升，反而还不如不使用 Softmax 激活函数的简单模型。

问题的根源不在于 Softmax 激活函数本身，而在于我们衡量误差的方式：**均方误差损失函数**。

均方误差损失函数是根据预测值和标签值的平方误差来衡量模型预测的差距，并据此给出不同的梯度来调整模型参数。损失值越大、梯度也越大。

但是 Softmax 激活函数将实际预测值转换成了相对的概率分布。

当预测值远离标签值的时候，Softmax 激活函数的结果非常接近于 0 或者 1，曲线变得非常平缓，因此给出的概率分布变化很小，出现**梯度饱和**现象。这相当于对于较大的预测误差，Softmax 激活函数有明显的弱化作用。这种情况下，继续使用均方误差损失函数，会造成梯度减弱，训练停滞不前。

---

为了解决这种“预测越错，修正越小”的逻辑矛盾，我们为分类任务引入一套新的评价标准：**交叉熵损失函数** (Cross Entropy Loss)。它能与Softmax 激活函数完美配合，在模型预测偏差较大时提供巨大的修正动力，来弥补 Softmax 激活函数的弱化效果，而在预测接近正确时则变得温和。

In [53]:
from abc import abstractmethod, ABC
import numpy as np

np.random.seed(99)

## 基础架构

### 张量

In [54]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    def size(self):
        return np.prod(self.data.shape[1:])

    def __str__(self):
        return f'Tensor({self.data})'

### 基础数据集

In [55]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size
        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    def shape(self):
        return Tensor(self.features).size(), Tensor(self.labels).size()

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size

        feature = Tensor(self.features[start: end])
        label = Tensor(self.labels[start: end])
        return feature, label

### 基础层

In [56]:
class Layer(ABC):

    def __init__(self):
        self.training = True

    def __call__(self, x: Tensor):
        return self.forward(x)

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def parameters(self):
        return []

    def __str__(self):
        return ''

### 基础损失函数

In [57]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### 基础优化器

In [58]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def reset(self):
        for p in self.parameters:
            p.grad = np.zeros_like(p.data)

    @abstractmethod
    def step(self):
        pass

### 基础模型

In [59]:
class Model(ABC):

    def __init__(self, layer, loss_fn, optimizer):
        self.layer = layer
        self.loss_fn = loss_fn
        self.optimizer = optimizer

    @abstractmethod
    def train(self, dataset, epochs):
        pass

    @abstractmethod
    def test(self, dataset):
        pass

## 数据

### MNIST 数据集

In [60]:
class MNISTDataset(Dataset):

    def __init__(self, filename, batch_size=1):
        self.filename = filename
        super().__init__(batch_size)

    def load(self):
        with (np.load(self.filename, allow_pickle=True) as f):
            self.train_features, self.train_labels = self.normalize(f['x_train'], f['y_train'])
            self.test_features, self.test_labels = self.normalize(f['x_test'], f['y_test'])

    @staticmethod
    def normalize(x, y):
        inputs = x / 255
        inputs = np.expand_dims(inputs, axis=1)
        targets = np.zeros((len(y), 10))
        targets[range(len(y)), y] = 1
        return inputs, targets

    def estimate(self, predictions):
        count = (predictions.data.argmax(axis=1) == self.labels.argmax(axis=1)).sum()
        total = len(self.labels)
        return count / total

## 模型

### 线性层

In [61]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        super().__init__()
        self.weight = Tensor(np.random.rand(out_size, in_size) / in_size)
        self.bias = Tensor(np.random.rand(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad.T @ x.data
            self.bias.grad += np.sum(p.grad, axis=0)
            x.grad += p.grad @ self.weight.data

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def parameters(self):
        return [self.weight, self.bias]

    def __str__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

### 顺序层

In [62]:
class Sequential(Layer):

    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def train(self):
        for l in self.layers:
            l.train()

    def eval(self):
        for l in self.layers:
            l.eval()

    def forward(self, x: Tensor):
        for l in self.layers:
            x = l(x)
        return x

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

    def __str__(self):
        return '\n'.join(str(l) for l in self.layers if str(l))

### 展平层

In [63]:
class Flatten(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.array(x.data.reshape(x.data.shape[0], -1)))

        def gradient_fn():
            x.grad += p.grad.reshape(x.data.shape)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __str__(self):
        return f'Flatten[]'

### 丢弃层

In [64]:
class Dropout(Layer):

    def __init__(self, dropout_rate=0.2):
        super().__init__()
        self.dropout_rate = dropout_rate

    def forward(self, x: Tensor):
        if not self.training:
            return x

        mask = np.random.random(x.data.shape) > self.dropout_rate
        p = Tensor(x.data * mask)

        def gradient_fn():
            x.grad += p.grad * mask

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __str__(self):
        return f'Dropout[rate={self.dropout_rate}]'

### ReLU 激活函数

In [65]:
class ReLU(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.maximum(0, x.data))

        def gradient_fn():
            x.grad += p.grad * (p.data > 0)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __str__(self):
        return f'ReLU[]'

### Tanh 激活函数

In [66]:
class Tanh(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.tanh(x.data))

        def gradient_fn():
            x.grad += p.grad * (1 - p.data ** 2)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __str__(self):
        return f'Tanh[]'

### Sigmoid 激活函数

In [67]:
class Sigmoid(Layer):

    def __init__(self, clip_range=(-100, 100)):
        super().__init__()
        self.clip_range = clip_range

    def forward(self, x: Tensor):
        z = np.clip(x.data, self.clip_range[0], self.clip_range[1])
        p = Tensor(1 / (1 + np.exp(-z)))

        def gradient_fn():
            x.grad += p.grad * p.data * (1 - p.data)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __str__(self):
        return f'Sigmoid[]'

### Softmax 激活函数

In [68]:
class Softmax(Layer):

    def __init__(self, axis=-1):
        super().__init__()
        self.axis = axis

    def forward(self, x: Tensor):
        exp = np.exp(x.data - np.max(x.data, axis=self.axis, keepdims=True))
        p = Tensor(exp / np.sum(exp, axis=self.axis, keepdims=True))

        def gradient_fn():
            grad = np.sum(p.data * p.grad, axis=self.axis, keepdims=True)
            x.grad += p.data * (p.grad - grad)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __str__(self):
        return f'Softmax[]'

### 损失函数（均方误差）

In [69]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data) / len(y.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### 损失函数（交叉熵）

交叉熵的核心思想是：

* 预测概率越接近标签值，损失值越小；
* 预测概率越偏离真实标签值，损失值呈指数级增长。

**前向传播**：交叉熵损失的定义是:

$$
L = -\sum_{i=1}^{n} y_i \log(a_i)
$$

这里：
* $y_i$：标签值的概率（独热编码）
* $a_i$：激活值的类别概率（Softmax 激活函数的输出值）

这里的 log 运算当概率接近 1 时变化缓慢，而在概率接近 0 时变化剧烈。可以说是 Softmax 激活函数的指数运算的“天敌”。当两者结合进行求导时，原本在 Softmax 饱和区消失的梯度，会被 log 函数重新放大。

**梯度计算**：交叉熵损失函数的实现，通常将 Softmax 激活函数的计算包含在内。这样做的目的，就是可以大大简化梯度计算，公式为：

$$
\delta = p_i - y_i
$$

这里：
* $y_i$：标签值的概率（独热编码）
* $p_i$：（无 Softmax 激活函数的）预测值

**父节点列表**（parents）：（无 Softmax 激活函数的）预测值。

**参数列表**（parameters）：无。

<div style="border-left: 4px solid #4CAF50; background:#f9f9f9; padding:10px; margin:10px 0;">
<strong>💡 重要：</strong> 这个交叉熵损失函数已经包括 Softmax 激活函数的计算在内。因此，使用的时候就不再需要输出层激活函数。
</div>

In [70]:
class CELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        exp = np.exp(p.data - np.max(p.data, axis=-1, keepdims=True))
        softmax = exp / np.sum(exp, axis=-1, keepdims=True)

        log = np.log(np.clip(softmax, 1e-10, 1))
        ce = Tensor(0 - np.sum(y.data * log) / len(y.data))

        def gradient_fn():
            p.grad += (softmax - y.data) / len(y.data)

        ce.gradient_fn = gradient_fn
        ce.parents = {p}
        return ce

### 损失函数（二元交叉熵）

在处理二元分类问题时，我们通常只关心一个预测概率。这时我们就不能再使用 Softmax **向量**激活函数了。而是使用 Sigmoid 激活函数将输出映射到 (0, 1) 的区间里。

**前向传播**：二元交叉熵损失的定义是：

$$
L = -(y \log(a) + (1 - y) \log(1 - a))
$$

这里：
* $y$：标签值的概率（0 或者 1）
* $a$：激活值的概率（Sigmoid 激活函数的输出值）

这个公式衡量预测概率与标签值之间的差异，当 $a$ 越接近 $𝑦$ 时损失值越小。

**梯度计算**：对激活值 $a$ 求导：

$$
\delta = \frac{a - y}{a (1 - a)}
$$

这里：
* $y$：标签值的概率（0 或者 1）
* $a$：激活值的概率（Sigmoid 激活函数的输出值）

二元交叉熵损失函数总是和 Sigmoid 激活函数配合使用。这个公式和 Sigmoid 激活函数的梯度公式正好相抵消，从而解决 Sigmoid 激活函数在遇到大的预测值时梯度饱和的问题。

**父节点列表**（parents）：激活值。

**参数列表**（parameters）：无。


In [71]:
class BCELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        clipped = np.clip(p.data, 1e-7, 1 - 1e-7)
        bce = Tensor(-np.mean(y.data * np.log(clipped) + (1 - y.data) * np.log(1 - clipped)))

        def gradient_fn():
            p.grad += (clipped - y.data) / (clipped * (1 - clipped)) / len(y.data)

        bce.gradient_fn = gradient_fn
        bce.parents = {p}
        return bce

### 优化器（随机梯度下降）

In [72]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

### 神经元网络模型

In [73]:
class NNModel(Model):

    def train(self, dataset, epochs):
        self.layer.train()
        dataset.train()

        for epoch in range(epochs):
            for i in range(len(dataset)):
                features, labels = dataset[i]

                predictions = self.layer(features)
                loss = self.loss_fn(predictions, labels)

                self.optimizer.reset()
                loss.backward()
                self.optimizer.step()

    def test(self, dataset):
        self.layer.eval()
        dataset.eval()

        features, labels = dataset.items()
        predictions = self.layer(features)
        loss = self.loss_fn(predictions, labels)
        return predictions, loss

## 设置

### 学习率

In [74]:
LEARNING_RATE = 0.01

### 批大小

In [75]:
BATCH_SIZE = 2

### 轮次

In [76]:
EPOCHS = 10

## 训练

### 迭代

让我们尝试使用一下交叉熵损失函数，看看训练效果会如何。

这里要记得去掉输出层的 Softmax 激活函数，因为我们的交叉熵损失函数已经包含 Softmax 激活函数的计算了。

In [77]:
dataset = MNISTDataset('tinymnist.npz', BATCH_SIZE)
layer = Sequential([Flatten(),
                    Linear(dataset.shape()[0], 64),
                    ReLU(),
                    Dropout(),
                    Linear(64, dataset.shape()[1])])
loss_fn = CELoss()
optimizer = SGDOptimizer(layer.parameters(), lr=LEARNING_RATE)

model = NNModel(layer, loss_fn, optimizer)
model.train(dataset, EPOCHS)
print(layer)

Flatten[]
Linear[weight(64, 784); bias(64,)]
ReLU[]
Dropout[rate=0.2]
Linear[weight(10, 64); bias(10,)]


## 验证

### 测试

In [78]:
predictions, loss = model.test(dataset)
accuracy = dataset.estimate(predictions)
print(f'accuracy: {accuracy:.2%}')

accuracy: 89.80%


相较于使用 Softmax 激活函数配合均平误差损失函数，改用（无 Softmax 激活函数的）交叉熵损失函数，确实显著提升了模型的准确率。

但是，在当前这个相对简单的数据集上，我们并没有看到比之前的简单模型效果更好。这主要是因为数据分布较为简单，类别间区分度高，使得复杂的概率建模（比如：交叉熵损失函数）带来的优势不明显，反而导致了训练变缓。

## 总结

| 任务类型        |  输出层激活函数  | 推荐损失函数 |                                 备注 |
|-------------|:---------:|-------:|-----------------------------------:|
| 回归问题（预测数值）  | 无 / ReLU  |  平均平方差 |                                    |
| 二元分类问题（是非题） |  Sigmoid  |  二元交叉熵 |                                    |
| 多元分类问题（多选一） |  Softmax  |    交叉熵 | 交叉熵已包含 Softmax<br/>无需再单独使用 Softmax |

## 课后练习

随着网络模型越来越复杂，超参数的选择也越来越重要。尝试调整学习率、轮次、隐藏层神经元个数等，看看会怎样影响那准确率。